<a href="https://colab.research.google.com/github/greapis/Python_practice/blob/main/Tensorflow_AI_LS_CIFAR_10(CNN).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

CIFAR-10 Datasets은 제공됨
 - MNIST : from tensorflow.keras.datasets import mnist
 - CIFAR-10 Datasets 역시 keras에서 기본적으로 제공

In [2]:
# Road library

import tensorflow as tf
from tensorflow.keras.datasets import cifar10
  #  keras에서 Datasets 제공

from tensorflow.keras import layers, models
from tensorflow.keras.utils import to_categorical
  # using "One-Hot Encoding"

import numpy as np
from PIL import Image

Data 전처리
 - X : 정규화(MNIST와 동일)

     각 픽셀은 0과 255 사이의 정수값으로 설정됨
     해당 값들을 0과 1로 정규화하는 것이 필요

 - Y : Label Encoding : One-Hot Encoding
     각 클래스를 독립적인 카테고리로 인식하기 위함
         airplaine  = [1,0,0,0,0,0,0,0,0,0]
         automobile = [0,1,0,0,0,0,0,0,0,0]   

In [4]:
# Data 전처리

# Roding CIFAR-10 datasets
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
  # x : pixels value(0~255),
  # y : labels (airplane, bird 등)

# Ont-Hot Encoding 적용
y_train = to_categorical(y_train)
y_test = to_categorical(y_test)

# Nomalization data
x_train, x_test = x_train / 255.0, x_test / 255.0

모델 구축 : 1번 합성곱 + 풀링 layer

 - 합성곱 layer(Convolutional Layer)

   필터수, 커널 크기(커널 = filter), 활성화 함수 설정

   input_shape = (32,32,3)인 이유는 RGB가 포함되었기 때문
   

 - Pooling Layer : Max Pooling

   Pooling filter size : (2 X 2)

   Max Pooling : (2 X 2) 블록 내에서 가장 큰 값을 선정


In [6]:
# CNN Model : Convolutional + Pooling Layers

model = models.Sequential()
model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(32,32,3)))
  #  Conv2D(32, (3, 3))) 32:필터 수, (3, 3) : 커널 크기

model.add(layers.MaxPooling2D(2,2))
  #  MaxPooling2D(2,2) : Pooling filer 크기 : (2,2)



/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
